# Tutorial on connecting DeepPavlov bot to Yandex.Alice
In this tutorial we show how to develop a simple bot and connect it to Yandex.Alice. Our bot is able to greet, say goodbye, and answer questions based on an FAQ list. In our case we take typical questions of MIPT entrant as an example. This list consists of 15 questions with corresponding answers with manually generated 4-6 paraphrases for each question. More details on how to build bots with DeepPavlov can be found [here](https://medium.com/deeppavlov).

## Requirements

To connect your bot to Yandex.Alice you need to have a dedicated IP address. You also need to create an ssl certificate and a key (more details in corresponding section).

First, create a virtual env with python3.6 (`source activate py36`). Then download DeepPavlov

In [ ]:
!pip install -q deeppavlov

## Implementation

Create skills for greeting, saying goodbye, and fallback (in case a bot doesn't have an appropriate answer).

In [ ]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill

hello = PatternMatchingSkill(responses=['Привет!', 'Приветик', 'Здравствуйте', 'Добрый день'],
	patterns=['Привет', 'Здравствуйте', 'Добрый день'])
bye = PatternMatchingSkill(responses=['Пока!', 'До свидания, надеюсь смог вам помочь', 'До встречи!'],
	patterns=['До свидания', 'Пока', 'Спасибо за помощь'])
fallback = PatternMatchingSkill(responses=['Я не понял, но могу попробовать ответить на другой вопрос',
	'Я не понял, задайте другой вопрос'], default_confidence = 0.5)

Download '.csv' table with questions and answers

In [ ]:
import pandas as pd
faq_table = pd.read_csv('http://files.deeppavlov.ai/faq/school/faq_school.csv')
# You may also specify path to a local file instead of a link
faq_table[:10]

There is a skill, that can recognize user intents and respond with a corresponding message. It's perfectly suitable for an FAQ task. It has the following arguments:
1. `data_path` - path/link to '.csv' table
2. `x_col_name` - name of the questions column in the table
3. `y_col_name` - name of the answers column in the table
4. `edit_dict` - dict of edits of the skill configuration file
5. `save_load_path` - path, where models will be saved or loaded from
6. `train` - should model be trained or not

In [ ]:
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill

faq = SimilarityMatchingSkill(data_path = 'http://files.deeppavlov.ai/faq/school/faq_school.csv',
                              x_col_name = 'Question', 
                              y_col_name = 'Answer',
                              save_load_path = './faq_school',
                              config_type = 'tfidf_autofaq',
                              train = True)

This code takes an existing model and trains it with your data. Once you've trained your model, you can load it without trainig, simply writing `faq = SimilarityMatchingSkill(load_path='./faq_doctor')`. `SimilarityMatchingSkill` is a simple wraper of configuration files, that describes intent recognition models. `edit_dict` contains the part that should be changed in the configuration file. More information about the configuration files you can find [here](http://docs.deeppavlov.ai/en/0.1.6/intro/config_description.html).

The last step is to bring all skills together using Agent.

In [ ]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

agent = DefaultAgent([hello, bye, faq, fallback], skills_selector=HighestConfidenceSelector())

Check that everything works correctly. Agent is able to receive a list of messages.

In [ ]:
print(agent(['Привет', 'Мне нужен личный кабинет', 'Пока', 'бла бла бла']))

In order to connect to Alice we need an ssl key and a certificate. Replace `<MACHINE_IP_ADDRESS>` with an actual IP address or a domain name of your server.

In [ ]:
!openssl req -new -newkey rsa:4096 -days 365 -nodes -x509 -subj "/CN=<MACHINE_IP_ADDRESS>" -keyout my.key -out my.crt

In [ ]:
from utils.alice import start_agent_server

start_agent_server(agent, host='0.0.0.0', port=5000, endpoint='/faq', ssl_key='my.key', ssl_cert='my.crt')

Now you can specify webhook (`<MACHINE_IP_ADDRESS>:5000/faq`) in [Yandex.Alice skill settings](https://dialogs.yandex.ru/developer/) and test it!


# Useful links

[DeepPavlov repository](https://github.com/deepmipt/DeepPavlov)

[DeepPavlov demo page](https://demo.ipavlov.ai)

[DeepPavlov documentation](https://docs.deeppavlov.ai)

[DeepPavlov blog](https://medium.com/deeppavlov)